# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "../WeatherPy/cities.csv"
vacation_df = pd.read_csv(csv_path)
vacation_df.head()
# vacation_df.info()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mirny,62.5353,113.9611,-26.50,97,100,1.09,RU,1645069569
1,1,Geraldton,-28.7667,114.6000,40.23,10,0,10.29,AU,1645069569
2,2,Cabo San Lucas,22.8909,-109.9124,21.61,63,33,5.65,MX,1645069319
3,3,Cayenne,4.9333,-52.3333,26.16,84,75,1.54,GF,1645069547
4,4,Nang Rong,14.6377,102.7914,28.20,62,60,3.24,TH,1645069570


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Reference - Class 3 - 07-Stu_Airport_Map

# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = vacation_df[["Lat", "Lng"]]

# Convert from int to float
humidity = vacation_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=5,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Conditions:
#  A max temperature lower than 40 degrees but higher than -33.
#  Wind speed less than 20 mph.
#  Zero cloudiness.
# Reference: https://keytodatascience.com/selecting-rows-conditions-pandas-dataframe/
     
weather_criteria_df = pd.DataFrame(vacation_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

weather_con_df = weather_criteria_df[(weather_criteria_df["Max Temp"] > -33) & (weather_criteria_df["Max Temp"] < 40) & (weather_criteria_df["Wind Speed"] < 20) & (weather_criteria_df["Cloudiness"] > 0)]
weather_con_df.head()


,City,Max Temp,Wind Speed,Cloudiness
0,Mirny,-26.50,1.09,100
2,Cabo San Lucas,21.61,5.65,33
3,Cayenne,26.16,1.54,75
4,Nang Rong,28.20,3.24,60
5,New Norfolk,24.40,3.13,2


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Dataframe
hotel_df = pd.DataFrame(vacation_df, columns = ["City", "Country", "Lat", "Lng"])

# Set up additional column to hold hotel information
hotel_df["Hotel Name"] = ""

hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,Mirny,RU,62.5353,113.9611,
1,Geraldton,AU,-28.7667,114.6000,
2,Cabo San Lucas,MX,22.8909,-109.9124,
3,Cayenne,GF,4.9333,-52.3333,
4,Nang Rong,TH,14.6377,102.7914,


In [30]:
# Reference - Code is from Day 3 class - 05-Stu_Google_Complex - 01:59:39

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "names": "hotel",
    "key": g_key
}


# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_hotel = requests.get(base_url, params=params)

    # convert to json
    name_hotel = name_hotel.json()
    print(json.dumps(name_hotel, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_hotel["results"][1]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

{
    "html_attributions": [],
    "next_page_token": "Aap_uEBXZ1ll-y8Fj6n2bY1tGt2XBJF09n2ZPIq7tFUTr2nWJ4WtjW_vNSccfEiI3RK6TFKTHzZYAGNlB3yOSq8v-3UUj5-gdf4lCghU4NicqxJA3EYfQljViBA2PRuCJ8RnsCCPIFbBgX0zjt-mR-2iKciNbXSn6493EYm3sN3ZOTAi9NsFGklLQ55IaW_k22FMTiTtYquRSBYyR4GQwrbHTeJ9gt3LsU9V-pM33jXAUPnlQETuzNWM4W4i7f-vjF-pYcRQxz43Y-_IjWgNh28Ldw956X27mtgqZnlhDBCWXkvt4-d-Dym4OrOBkTkidRXlFQJV6QBYumNlhN69OsNYOfwCZbSHL0wAgpuM3Gbrfp7sukgeR29SINRCXXoqaiLFHGtuCI0Bm_8GDv4p49H2ZoYplFcVy0skM4_FNbL8K-HLpPLkJAvRLg",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 62.5313374,
                    "lng": 113.9773881
                },
                "viewport": {
                    "northeast": {
                        "lat": 62.56367604612313,
                        "lng": 114.044519006685
                    },
                    "southwest": {
                        "lat": 62.50793897525757,
                        "lng": 113.9343518420336


{
    "html_attributions": [],
    "next_page_token": "Aap_uEDmSLiVLPBlsY3AD_MQS84B7yuNMEgES6mal4nG8Qhjp54Ufh4qT5_x2GHlnjf5XK85bgcdvytnawyViIyQdlpaoriQ9o1xEvHDT4lcKSRgLgkSCzgZO9Uq6mD8Zl0pn6_O1V87Al17zXVdMB2oZnwUFWfcA3ecGuopzb41xF_xCKb9rt9AysRYkC7UQ4AsOX-uhBoZqiIFCti_wFwJHGRanhrcbsEqPcBRgT8xuVqgH5OjUqZn5TC6P5r-lc7pvMxDaxmteawRPIPpsjQZCK8PWhXHixrLD5pN6Avxlqywgz8V8Y3f4gd_l8biAZ5wjexvUeDGWj6cTEb-Sx4jZ1jE7V4ZVR54xObqEbV7uYC60_GbLN3qzrIGR-0yH7jIdCqnhdl_tqZWmUgEYftMd8JAf-C1m9GUtILql6T17Yn27dELThuVRQ",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -28.7761346,
                    "lng": 114.611295
                },
                "viewport": {
                    "northeast": {
                        "lat": -28.76320798509792,
                        "lng": 114.6261427696306
                    },
                    "southwest": {
                        "lat": -28.7886324981025,
                        "lng": 114.596259950611

{
    "html_attributions": [],
    "next_page_token": "Aap_uEDJsl9xqTt4V7AI82B4CVOIR-woPAZuPKURO3JBpGA5xuju9t6QUY6QFhYjd3BO0GevHeq8WhXy0nZTaYUJosmCx0zmkDqSXEUcsD9VjU03s2WM97kxxCdUREoco2sKAqKW8sxBEgU1HgUyC_qggJ0WS88k0l6zfH6WM8iNtjq1fkRVxRQLxYilfN1j-nNNysOkPHk3egr7QF5-1JteI4z7ucIUDTu7khukDOXxD3X-PR4DZ5FSKYOE1G8-93uHIMFFr18RBfJdTwD9noCYLMsV4Sw8pE4XN0eouYQ18pJdkvv49ZoJlyGLQPOnnFxfVxgGXSGPTCvF7P__-AZjsqeiLN15-vWW4GNltGjhwVUgwZBND_X5hdgU0EAS7yhJGGpXI-4wAWSVr3sDR7b2VgsIcTKqUzZSpw0KLB_-BXLMdcrHqm1lrA",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 22.8905327,
                    "lng": -109.9167371
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.92028757787479,
                        "lng": -109.8942558770843
                    },
                    "southwest": {
                        "lat": 22.87216499597992,
                        "lng": -109.9717402828

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAuObyehvD73tsPOEZSbU9fsIAaOS7lrHmjJfQEuWpBnuHYJFRk9kVlSdg-RuyX4T-zR17wjQdwqIe8Uri0Iw48ohU0ef0td35VIUpFOWRu-M24xSYiE8by6IBPvA2cK2tna7bQCa2B3Etcp2oUjPQKvzV0SeLhfnUmjrMVQ4eE_Qppf1gFQd22tcsLCXZ4EjAWe4CBkqwhmdjN_JgQOj17Oe8NcFJMaj7yLMhKjsVxtx79IHpg-G5UTbsUWa2prMgCyyJ9o7KSuS_D-HQ4SJ490S2EoWLUvfXVaq7XeTLSo-_41roYP-iZKOQ3QqrYdVQMABmvwFoYxaPgR_ceRFqHhTV4u-htn9ihB2FiwxgdCe5XGVh9TzarNAi88f6RK8e-wUiaMdCFrIDIiYbkEl6KIuFzM0iZqxTAEgr8ZcHoojuheQSoYQ",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 4.92242,
                    "lng": -52.3134531
                },
                "viewport": {
                    "northeast": {
                        "lat": 4.951705390698261,
                        "lng": -52.2801374948585
                    },
                    "southwest": {
                        "lat": 4.88767638044199,
                        "lng": -52.34268668281687
  

{
    "html_attributions": [],
    "next_page_token": "Aap_uECbs_GFTM38Jnm23eqUYRPlMf0CYbcwGrB2MZUEDGhG0UAbSDUrDLITDqs3AJdHWlWozOiVnF2dAkUJejf6d4Gci3cYeQkEnE0QjXEZ5-DKaOoKVCl3PBks-XP5117ufRdUSXN0IrWq7X5IuVzLwgDZlAy9NlyakKh3vBlXwcYNQ0TLi4GNQ3r19c3IDvNLORBYPNoM-p1KsHU00IFb1WVPafvLiZ-ZZgQ12uyC-kD0oaWdygVBT5Z-0emDuufX5CJMAnUK1sOIjVALaEm8KPJw_SFvGwdTgHHGKUPMmVri3zkbfM-jReAwN9O0XSntPagcOPRJUB7N-ktoutbm4vNvqcuDfUm18YRvBt4rfvPDlKifJe6QCR34zICRXEaYzPZZPFd31Lw3UhNMqHPJlnV-ELuOvxo1D61PrR-L7VzCmhHoBRe5VQ",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 14.6335733,
                    "lng": 102.7934934
                },
                "viewport": {
                    "northeast": {
                        "lat": 14.71497882256561,
                        "lng": 102.8282298502953
                    },
                    "southwest": {
                        "lat": 14.59970953709549,
                        "lng": 102.742863465447


{
    "html_attributions": [],
    "next_page_token": "Aap_uECiBYKFlbN3hkhRzjzgGS9Pb-DwxBbVqRN9QV05F2uL_xOW1QbALbvezKLmH481tw8IX3cchCKlbN6P3RkShi470R4aH7eoDl4UbqR0uMMRonwcr6Rj0GopO_Ndueg3REFpRDB0hgUjgAcd0DdSZ9zPd8Qok7w1Zf-dbEs-2wRqH3eTzZN64VIYzy-2PWpUMb2CFcQSxjj4-Vj344oWqvacrKxhym9u0WrRXV-tXvR02DNcj4G8d26LtZZteQTyDwfIoDZHpLmAl3-9FkZFgSW0h4fll8q38FtlJ-F4AkUZQI9XID70XjvPmN_BsCUUp4M1SzpfyD2b_j5Mo0LMAIz3Vkk5hMZqRwpbS418W_j8EUkS-nEOBLXfrng2if7OJM75-Z9xx6IHrddu4tMI6oYtLnSxLkbsZ2az4OPAlfwXWrJnMuwy4g",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -42.781144,
                    "lng": 147.0622665
                },
                "viewport": {
                    "northeast": {
                        "lat": -42.75857608597855,
                        "lng": 147.0920986679486
                    },
                    "southwest": {
                        "lat": -42.8056163856086,
                        "lng": 147.019694690057

In [31]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,Mirny,RU,62.5353,113.9611,квартира
1,Geraldton,AU,-28.7667,114.6000,Broadwater Mariner Resort
2,Cabo San Lucas,MX,22.8909,-109.9124,Hotel Tesoro Los Cabos
3,Cayenne,GF,4.9333,-52.3333,Hôtel Le Dronmi
4,Nang Rong,TH,14.6377,102.7914,Phanomrungpuri Boutique Hotels & Resorts


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [33]:
marker_locations = hotel_df[["Lat", "Lng"]]

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
fig.add_layer(markers)
fig

TraitError: The 'layers' trait of a Map instance contains an Instance of a List which expected a Widget, not the DataFrame        Lat       Lng
0  62.5353  113.9611
1 -28.7667  114.6000
2  22.8909 -109.9124
3   4.9333  -52.3333
4  14.6377  102.7914
5 -42.7826  147.0587.